In [31]:
import pandas as pd
import numpy as np
from itertools import combinations, product

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from xgboost import XGBClassifier
from sklearn import metrics
import time

In [2]:
df = pd.read_csv('../../Dataset/CIC-AndMal2017/full-2017.csv')

C:\Users\admin\AppData\Local\Temp\ipykernel_15796\1196048977.py:1: DtypeWarning: Columns (48,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../Dataset/CIC-AndMal2017/full-2017.csv')


In [3]:
df.shape

(353886, 86)

In [4]:
df_cp = df.copy()

In [5]:
def clean_df(df):
    # Remove the space before each feature names
    df.columns = df.columns.str.strip()
    print('dataset shape', df.shape)

    # This set of feature should have >= 0 values
    num = df._get_numeric_data()
    num[num < 0] = 0

    zero_variance_cols = []
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(zero_variance_cols, axis = 1, inplace = True)
    print('zero variance columns', zero_variance_cols, 'dropped')
    print('shape after removing zero variance columns:', df.shape)

    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    print(df.isna().any(axis = 1).sum(), 'rows dropped')
    df.dropna(inplace = True)
    print('shape after removing nan:', df.shape)

    # Drop duplicate rows
    df.drop_duplicates(inplace = True)
    print('shape after dropping duplicates:', df.shape)

    column_pairs = [(i, j) for i, j in combinations(df, 2) if df[i].equals(df[j])]
    ide_cols = []
    for column_pair in column_pairs:
        ide_cols.append(column_pair[1])
    df.drop(ide_cols, axis = 1, inplace = True)
    print('columns which have identical values', column_pairs, 'dropped')
    print('shape after removing identical value columns:', df.shape)
    return df
df_cp = clean_df(df_cp)

dataset shape (353886, 86)
zero variance columns [] dropped
shape after removing zero variance columns: (353886, 86)
3 rows dropped
shape after removing nan: (353883, 86)
shape after dropping duplicates: (353883, 86)
columns which have identical values [('Total Backward Packets', 'Subflow Bwd Packets'), ('Total Length of Fwd Packets', 'Subflow Fwd Bytes'), ('Total Length of Bwd Packets', 'Subflow Bwd Bytes'), ('Fwd PSH Flags', 'SYN Flag Count'), ('Bwd PSH Flags', 'Fwd URG Flags'), ('Bwd PSH Flags', 'Bwd URG Flags'), ('Bwd PSH Flags', 'RST Flag Count'), ('Bwd PSH Flags', 'ECE Flag Count'), ('Bwd PSH Flags', 'Fwd Avg Bytes/Bulk'), ('Bwd PSH Flags', 'Fwd Avg Packets/Bulk'), ('Bwd PSH Flags', 'Fwd Avg Bulk Rate'), ('Bwd PSH Flags', 'Bwd Avg Bytes/Bulk'), ('Bwd PSH Flags', 'Bwd Avg Packets/Bulk'), ('Bwd PSH Flags', 'Bwd Avg Bulk Rate'), ('Fwd URG Flags', 'Bwd URG Flags'), ('Fwd URG Flags', 'RST Flag Count'), ('Fwd URG Flags', 'ECE Flag Count'), ('Fwd URG Flags', 'Fwd Avg Bytes/Bulk'), ('Fwd

In [6]:
df_cpz = df_cp.copy()

In [7]:
# df_cp = df_cpz.copy()

In [8]:
df_cp.loc[df_cp.Label != 'BENIGN', 'label'] = 1
df_cp.loc[df_cp.Label == 'BENIGN', 'label'] = 0

In [9]:
drop_columns = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Protocol', 'Label']
df_cp = df_cp.drop(drop_columns, axis=1)

In [10]:
df_cp.shape

(353883, 64)

In [11]:
df_cp = df_cp.drop(['Unnamed: 0'], axis=1)

In [12]:
df_cp.columns

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', '

In [16]:
from sklearn.feature_selection import SelectKBest


def select_k_best(score, X, Y):
    selector = SelectKBest(score, k=62)
    selector.fit_transform(X, Y)
    names = X.columns.values[selector.get_support()]
    scores = selector.scores_[selector.get_support()]
    names_scores = list(zip(names, scores))
    df_reduced = pd.DataFrame(data=names_scores, columns=['feature_names', 'score'])

    df_reduced = df_reduced.sort_values(['score', 'feature_names'], ascending=[False, True])
    print(df_reduced)
    return df_reduced

In [17]:
from sklearn.feature_selection import mutual_info_classif, f_classif

frs_mi = select_k_best(mutual_info_classif, df_cp[df_cp.columns.difference(['label'])], df_cp.label)

             feature_names     score
47  Init_Win_bytes_forward  0.029856
0           ACK Flag Count  0.029792
21           Down/Up Ratio  0.018975
50          PSH Flag Count  0.018382
51      Packet Length Mean  0.015404
..                     ...       ...
14           Bwd PSH Flags  0.000000
20          CWE Flag Count  0.000000
36           Fwd PSH Flags  0.000000
45                Idle Std  0.000000
61    min_seg_size_forward  0.000000

[62 rows x 2 columns]


In [20]:
features = frs_mi[frs_mi.score >= 0.01].feature_names
features

47         Init_Win_bytes_forward
0                  ACK Flag Count
21                  Down/Up Ratio
50                 PSH Flag Count
51             Packet Length Mean
16         Bwd Packet Length Mean
5             Average Packet Size
6            Avg Bwd Segment Size
52              Packet Length Std
18          Bwd Packet Length Std
53         Packet Length Variance
40          Fwd Packet Length Std
38         Fwd Packet Length Mean
57    Total Length of Bwd Packets
58    Total Length of Fwd Packets
7            Avg Fwd Segment Size
46        Init_Win_bytes_backward
37          Fwd Packet Length Max
48              Max Packet Length
Name: feature_names, dtype: object

In [23]:
X = df_cp[features]
# X = df_test.drop(['Label'], axis=1)
y = df_cp.label

Scaler

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [35]:
def test_ml(X, y):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, stratify = y, random_state = 0, test_size = 0.2)
    model = RandomForestClassifier(random_state=27)
    model.fit(X_train, y_train)
    
    tic = time.time()
    preds_val = model.predict(X_test)
    tac = time.time()
    print ("Test time: " + str(tac - tic))

    classification = metrics.classification_report(y_test, preds_val)
    confusion_matrix = metrics.confusion_matrix(y_test, preds_val)
    print(accuracy_score(y_test, preds_val))
    print(precision_score(y_test, preds_val))
    print(recall_score(y_test, preds_val))
    print(f1_score(y_test, preds_val))

In [36]:
test_ml(df_cp[features], df_cp.label)
test_ml(df_cp.drop(['label'], axis=1), df_cp.label)

Test time: 4.242176532745361
0.7927565581877266
0.8130060067902847
0.9598453583795835
0.8803445762951521
Test time: 5.9601545333862305
0.7788065746715019
0.8130808418669274
0.9368981547364926
0.8706092380254452
